In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'n2o': 3.2e-07, 'ch4': 1.8e-06}
band = [9]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
 

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.120502                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.058708
0.000750      2.0 -0.119356                  2.0 -0.057989
0.001052      3.0 -0.116891                  3.0 -0.056383
0.001476      4.0 -0.114367                  4.0 -0.054772
0.002070      5.0 -0.111823                  5.0 -0.053124
0.002904      6.0 -0.109294                  6.0 -0.051442
0.004074      7.0 -0.106842                  7.0 -0.049741
0.005714      8.0 -0.104449                  8.0 -0.047995
0.008015      9.0 -0.102183                  9.0 -0.046240
0.011243     10.0 -0.100100                 10.0 -0.044452
0.015771     11.0 -0.098301                 11.0 -0.042653
0.022122     12.0 -0.096895                 12.0 -0.040858
0.031031     13.0 -0.096065                 13.0 -0.039054
0.043528     14.0 -0.095186                 14.0 -0.036733
0.061057     15.0 -0.084306                 15.0 -0.027457
0.085645     16.0 -0.055524                 16.0 -0.005937
0.120136     17.0 -0.014390                 17.0  0.024753
0.168516     18.0  0.035145                 18.0  0.062623
0.236378     19.0  0.094252                 19.0  0.108836
0.331549     20.0  0.164610                 20.0  0.164848
0.465100     21.0  0.248696                 21.0  0.232281
0.652400     22.0  0.337965                 22.0  0.304280
0.915100     23.0  0.356815                 23.0  0.328343
1.283650     24.0  0.300332                 24.0  0.296887
1.800600     25.0  0.228455                 25.0  0.246529
2.525700     26.0  0.154988                 26.0  0.185778
3.542800     27.0  0.100046                 27.0  0.130571
4.969550     28.0  0.064671                 28.0  0.085977
6.970850     29.0  0.043115                 29.0  0.052812
9.778100     30.0  0.030150                 30.0  0.030669
13.715850    31.0  0.021765                 31.0  0.017760
19.239350    32.0  0.015375                 32.0  0.010600
26.987250    33.0  0.009643                 33.0  0.005517
37.855300    34.0  0.003901                 34.0 -0.000419
53.100050    35.0 -0.006345                 35.0 -0.012090
73.887500    36.0 -0.020203                 36.0 -0.028195
97.662500    37.0 -0.031638                 37.0 -0.041332
121.437500   38.0 -0.035660                 38.0 -0.046175
145.212500   39.0 -0.033615                 39.0 -0.044593
168.987500   40.0 -0.032046                 40.0 -0.042521
192.762500   41.0 -0.029954                 41.0 -0.039668
216.537500   42.0 -0.026726                 42.0 -0.035617
240.312500   43.0 -0.022003                 43.0 -0.030316
264.087500   44.0 -0.015741                 44.0 -0.023883
287.862500   45.0 -0.007946                 45.0 -0.015090
311.637500   46.0  0.001323                 46.0 -0.004970
335.412500   47.0  0.011842                 47.0  0.004707
359.187500   48.0  0.022895                 48.0  0.015076
382.962500   49.0  0.033000                 49.0  0.026243
406.737500   50.0  0.044397                 50.0  0.038481
430.512500   51.0  0.056614                 51.0  0.050231
454.287500   52.0  0.069315                 52.0  0.061954
478.062500   53.0  0.082177                 53.0  0.074876
501.837500   54.0  0.094785                 54.0  0.089586
525.612500   55.0  0.107009                 55.0  0.104831
549.387500   56.0  0.118478                 56.0  0.118936
573.162500   57.0  0.129157                 57.0  0.130729
596.937500   58.0  0.139232                 58.0  0.140535
620.712500   59.0  0.148945                 59.0  0.149660
644.487500   60.0  0.159106                 60.0  0.160150
668.262500   61.0  0.169942                 61.0  0.172813
692.037500   62.0  0.179510                 62.0  0.185535
715.812500   63.0  0.183190                 63.0  0.192996
739.587500   64.0  0.184146                 64.0  0.198845
76

# Fluxes by Layer

CRD                           CLIRAD                \
                         flug       fldg      fnetg       flug          fldg   
pressure     level                                                             
0.000000e+00 1     -13.104923   0.000000 -13.104923        NaN           NaN   
1.000000e-08 1            NaN        NaN        NaN -13.077350  9.047325e-08   
6.244000e-04 2     -13.104936   0.000004 -13.104932 -13.077356  2.466052e-06   
8.759000e-04 3     -13.104941   0.000005 -13.104936 -13.077359  3.435474e-06   
1.228600e-03 4     -13.104948   0.000007 -13.104941 -13.077363  4.829606e-06   
1.723400e-03 5     -13.104958   0.000011 -13.104947 -13.077368  6.833393e-06   
2.417400e-03 6     -13.104972   0.000015 -13.104957 -13.077375  9.713135e-06   
3.390900e-03 7     -13.104991   0.000022 -13.104969 -13.077385  1.385214e-05   
4.756500e-03 8     -13.105017   0.000031 -13.104986 -13.077399  1.979993e-05   
6.672000e-03 9     -13.105054   0.000044 -13.105010 -13.077419  2.834762e-05   
9.358900e-03 10    -13.105106   0.000063 -13.105043 -13.077446  4.062743e-05   
1.312780e-02 11    -13.105178   0.000091 -13.105087 -13.077483  5.826830e-05   
1.841450e-02 12    -13.105279   0.000130 -13.105149 -13.077535  8.360318e-05   
2.583020e-02 13    -13.105419   0.000185 -13.105234 -13.077608  1.199726e-04   
3.623230e-02 14    -13.105617   0.000264 -13.105352 -13.077708  1.721738e-04   
5.082340e-02 15    -13.105894   0.000377 -13.105517 -13.077847  2.475457e-04   
7.129060e-02 16    -13.106271   0.000550 -13.105721 -13.078031  3.649418e-04   
1.000000e-01 17    -13.106754   0.000844 -13.105910 -13.078253  5.671429e-04   
1.402710e-01 18    -13.107337   0.001359 -13.105979 -13.078493  9.251989e-04   
1.967600e-01 19    -13.107991   0.002247 -13.105744 -13.078704  1.554972e-03   
2.759970e-01 20    -13.108620   0.003761 -13.104859 -13.078781  2.653692e-03   
3.871000e-01 21    -13.108997   0.006305 -13.102693 -13.078511  4.553251e-03   
5.431000e-01 22    -13.108621   0.010524 -13.098097 -13.077475  7.810818e-03   
7.617000e-01 23    -13.106588   0.017243 -13.089345 -13.075000  1.321512e-02   
1.068500e+00 24    -13.102652   0.026274 -13.076378 -13.070868  2.101725e-02   
1.498800e+00 25    -13.097397   0.036328 -13.061069 -13.065450  3.073420e-02   
2.102400e+00 26    -13.091100   0.046366 -13.044734 -13.058945  4.185802e-02   
2.949000e+00 27    -13.084363   0.055171 -13.029191 -13.051813  5.335852e-02   
4.136600e+00 28    -13.077452   0.062335 -13.015117 -13.044296  6.421212e-02   
5.802500e+00 29    -13.070456   0.068101 -13.002355 -13.036539  7.342338e-02   
8.139200e+00 30    -13.063503   0.073082 -12.990421 -13.028761  8.026515e-02   
1.141700e+01 31    -13.056825   0.078111 -12.978714 -13.021353  8.476715e-02   
1.601470e+01 32    -13.050892   0.084032 -12.966860 -13.015072  8.815917e-02   
2.246400e+01 33    -13.046622   0.091508 -12.955114 -13.011374  9.256059e-02   
3.151050e+01 34    -13.045684   0.100902 -12.944781 -13.013059  1.001582e-01   
4.420010e+01 35    -13.050899   0.111982 -12.938917 -13.025403  1.118724e-01   
6.200000e+01 36    -13.073050   0.120753 -12.952296 -13.063169  1.241445e-01   
8.577500e+01 37    -13.132994   0.123799 -13.009195 -13.150717  1.322768e-01   
1.095500e+02 38    -13.225350   0.127053 -13.098298 -13.273809  1.389529e-01   
1.333250e+02 39    -13.342357   0.143631 -13.198726 -13.420875  1.559618e-01   
1.571000e+02 40    -13.473504   0.180108 -13.293396 -13.581023  1.905077e-01   
1.808750e+02 41    -13.616734   0.233087 -13.383647 -13.751575  2.412934e-01   
2.046500e+02 42    -13.771013   0.303006 -13.468006 -13.931056  3.090460e-01   
2.284250e+02 43    -13.937949   0.394673 -13.543276 -14.120768  3.984393e-01   
2.522000e+02 44    -14.118565   0.513322 -13.605243 -14.321377  5.136598e-01   
2.759750e+02 45    -14.314535   0.664960 -13.649576 -14.534339  6.593521e-01   
2.997500e+02 46    -14.527968   0.856015 -13.671954 -14.761489  8.440009e-01   
3.235250e+02 47    -14.761

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')